## Play-ground to make initial data distributions.

In [ ]:
#import Pkg; Pkg.add("FileIO")
using Distributions
using Plots
using FileIO
using JLD2
using LaTeXStrings
#Pkg.instantiate()

### This is a Maxwellinan distribution (3D) centered around v=0 and with m/kT = 1

In [ ]:
d  = LocationScale(0.0,1.,Chi(3))
data = rand(d,20000);
histogram(data)

The 1D distribution for $\frac{m}{kT} = 1$ is just the normal distribution centered at both $\pm v_p$. With $\sigma = 1$.


In [ ]:
N = 40000
vb = -10

data_n = [rand(Normal(-vb,1),N÷2); rand(Normal(vb,1),N÷2)]  #.+ rand(Normal(-1,1),20000)
histogram(data_n, bins=:scott, weights=repeat(1:80, outer=500))

### This is the distribution on the course, just to check that it is the same... 

I should apply some statistics here and check different momenta :-)

In [ ]:
"""The following routine returns a random velocity distributed on a double Maxwellian distribution function 
corresponding to two counter-streaming beams. The algorithm used to achieve this is called the rejection method, 
and will be discussed later in this course.

  // Function to distribute electron velocities randomly so as 
  // to generate two counter propagating warm beams of thermal
  // velocities unity and mean velocities +/- vb.
  // Uses rejection method.
"""
function distribution_stream(vb)
  #Generate random v value
  fmax = 0.5 * (1. + exp(-2.0 * vb * vb));
  vmin = - 5.0 * vb;
  vmax = + 5.0 * vb;
  v = vmin + (vmax - vmin) * (rand());

  #  Accept/reject value
  f = 0.5 * (exp(-(v - vb) * (v - vb) / 2.0) + exp(-(v + vb) * (v + vb) / 2.0))
  x = fmax * rand()
  if (x > f) return distribution_stream(vb)
  else return v
  end
end

In [ ]:
v = zeros(N)
for i in 1:N
    v[i] = distribution_stream(vb)
end

In [ ]:
histogram(v, bins=:scott, weights=repeat(1:80, outer=500))

At first sigth they look very similar....

We now make and save some distributions to use as standard ones.

In [ ]:
L = 100 #length of integration region
#N = 200000
N = 80000 #even number
vb = 3

r = L*rand(N);
vp = rand(Normal(vb,1),N÷2);
u = [r; -vp; vp];

#save("Initial_Distributions/par_dis_L100_N8_5_vp_3.jld2", "par_dis", u)

## Thermal Distributions

$$
f(x,v) = \frac{N}{L} \frac{1}{\sqrt{2\pi \theta}} e^{-\frac{v^2}{2\theta}}
$$

Thus, $\theta = \sigma^2$ is the variance of the distribution.

### Relativistic version

$$
f(x,v) = \frac{N}{L} \frac{1}{\sqrt{2\pi \theta}} e^{-\frac{v^2}{2\theta}}
$$

In [ ]:
L = 5 #length of integration region
#N = 200000
N = 80000 #even number
θ = 0.01

r = L*rand(N);
vp = rand(Normal(0,sqrt(θ)),N);
u = [r; vp];

In [ ]:
mean(vp)
var(vp)

In [ ]:
histogram(vp)

In [ ]:
save("Initial_Distributions/par_dis_L5_N8_5_theta01.jld2", "par_dis", u)
#save("Initial_Distributions/par_dis_L5_N2_5_theta001.jld2", "par_dis", u)
#save("Initial_Distributions/par_dis_L5_N2_5_theta0001.jld2", "par_dis", u)
#save("Initial_Distributions/par_dis_L5_N2_5_theta00001.jld2", "par_dis", u)


## Landau Damping ##

The initial distribution is: 

$$
f(x,v,\alpha, k, \theta) = \frac{1}{\sqrt{2\pi \theta}} e^{-\frac{v^2}{2\theta}} (1 + \alpha cos(kx) )
$$

Since it is a product we can split into two distributions one for the position and one for the velocity.
The one for the velocity we already have, so we need to make only the random sampling of the spatial one. 

Some definitions $\hat{k} = k / k_D = k * \sqrt{\theta}$ Or $k = \hat{k} / \sqrt{\theta}$ 

In these simulations, $\theta = 10^{-3}$ $\sqrt{\theta} = 0.03162$

Thus to $\hat{k} = 0.01$ corresponds a $\lambda = 2\pi * \sqrt{\theta} / \hat{k} = 19.869$

For $\hat{k} = 0.40$ corresponds a $\lambda = 0.497$

For the first case an $L = 2 \lambda = 39.738$ is used and $J = 1761*2 = 3522$, $dx = L/J = 0.0112$

For the second $L=15*\lambda = 7.455$ is used and $J = 62 * 15 = 930$, $dx = 0.0080$

In [ ]:
7.455 / (62 * 15)

In [ ]:
"""The following routine returns a random sample according to a given distribution function.
 The algorithm used to achieve this is called the rejection method.
 f is the distribution function and p are its parameters. We need also
 f_max which is a function of p, and the interval of the sampling region
 int = (x_min,x_max)
"""
function random_sampling_from_distribution(f,f_max,p,interval)
  (x_max,x_min) = interval
  fmax = f_max(p)
  x = x_min + (x_max - x_min) * (rand());
  #  Accept/reject value
  f_v = f(x,p)
  x_t = fmax * rand()
  if (x_t > f_v) return random_sampling_from_distribution(f,f_max,p,interval)
  else return x
  end
end

In [ ]:
damped = false
undamped = false
#damped = true
undamped = true

#N = 8_000_000
N = 800_000
#N = 80_000

if undamped
    L= 39.738 
    nm = 2
elseif damped
    L = 7.455
    nm = 15 # 12, 15, 19
end

k = 2π*nm/L # this here is k, not \hat{k}
α = 0.01 # 0.01
θ = 0.001 # 0.001
p = (α, k)

function f(x,p) 
    α, k = p
    return 1 + α *cos(k*x)
end

function f_max(p)
    α, k = p
    return 1+α
end

interval = (0., L)
r = zeros(N)
for i in 1:N
    r[i] = random_sampling_from_distribution(f,f_max,p,interval)
end

In [ ]:
using GR
Plots.histogram(r
#, bins=:scott
, bins = 300
#, weights=repeat(1:80, outer=1000)
,label = false
)

In [ ]:
vp = rand(Normal(0,sqrt(θ)),N);
u = [r; vp];

In [ ]:
#save("Initial_Distributions/par_dis_L40_N8_5_theta001_m1_am2.jld2", "par_dis", u)
#save("Initial_Distributions/par_dis_L40_N8_5_theta001_m2_am2.jld2", "par_dis", u)
#save("Initial_Distributions/par_dis_L40_N8_5_theta001_m4_am1.jld2", "par_dis", u)
#save("Initial_Distributions/par_dis_L40_N8_5_theta001_m4_am2.jld2", "par_dis", u)
#save("Initial_Distributions/par_dis_nodumping.jld2", "par_dis", u)
if damped 
    #save("Initial_Distributions/par_dis_8_4_damping.jld2", "par_dis", u)
    #save("Initial_Distributions/par_dis_8_5_damping.jld2", "par_dis", u)
    #save("Initial_Distributions/par_dis_8_5_abig_damping.jld2", "par_dis", u)
elseif undamped
    #save("Initial_Distributions/par_dis_8_5_undamping.jld2", "par_dis", u)
    #save("Initial_Distributions/par_dis_8_5_abig_undamping.jld2", "par_dis", u)
end

In [ ]:
N = 10^6
θ = 0.001
vp = rand(Normal(0,sqrt(θ)),N);
var(vp)

### Relativistic version

For the relativistic version we use the distribution (for momentum)

$$
f(p) = \frac{1}{4\pi m^3 c^3 \theta K^2(1/\theta)} e^{(1-\gamma) / \theta} \;\;\;\; \gamma = \sqrt{1+p^2}
$$

In [ ]:
Plots.plot(u -> f_p_rel(u,0.1)
, xscale=:log10, yscale=:log10, xlim=(0.0001,1)
, label=L"\theta = 0.1")
Plots.plot!(u -> f_p_rel(u,10.)
, xscale=:log10, yscale=:log10, xlim=(0.0001,100)
, label=L"\theta = 10.0")

### Relativistic distributions

#### Small momentum

If $p$ is small we have,

$$
\frac{(1 - \sqrt{1 + p^2})}{\theta} \approx -\frac{1}{2}\frac{p^2}{\theta}
$$

Therefore, for small $p$ we need to take an interval of about $p = \sqrt{10*2*\theta}$

If $p$ is larger than 1, then,

$$
\frac{(1 - \sqrt{1 + p^2})}{\theta} \approx -\frac{p}{\theta}
$$

Thus, in this case we need to take an interval of about $p = 1 + 10*\theta$

In [ ]:
damped = false
undamped = false
vlasov_exp_200 = false
vlasov_exp_40 = false
#damped = true
#undamped = true
#vlasov_exp_200 = true
vlasov_exp_40 = true

N_exp = 5
N = 8*10^(N_exp)

data_name = "par_dis_landau_rel_"

if undamped
    L= 39.738 
    m = 2
    data_name = data_name * "undamped_$(m)_"
elseif damped
    L = 7.455
    m = 15 # 12, 15, 19
    data_name = data_name * "damped_$(m)_"
elseif vlasov_exp_200
    L = 200
    m = 2
    data_name = data_name * "vla_200_$(m)_"
elseif vlasov_exp_40
    L = 40
    m = 2
    data_name = data_name * "vla_40_$(m)_"
end

k = 2π*m/L # this here is k, not \hat{k}
alpha_exp = 3
α = 10.0^(-alpha_exp) # 0.1 0.001

exp_Theta = 3
θ = 10.0^(-exp_Theta)
par_f_r = (α, k)

data_name = data_name * "alp$(alpha_exp)_8$(N_exp)_Th$(exp_Theta)"

interval = (0., L)
r = zeros(N)
for i in 1:N
    r[i] = random_sampling_from_distribution(f_r,f_r_max,par_f_r,interval)
end

p_max = 1 + 10*θ
#p_max = sqrt(20*θ)
int = (0, p_max)
vp = zeros(N÷2)

par_p = θ 

for i in 1:N÷2
    vp[i] = random_sampling_from_distribution(f_p_rel,f_p_rel_max,Base.parseint_preamble,int)
end
u = [r; vp; -vp];

In [ ]:
histogram(r)

In [ ]:
histogram([vp;-vp], label="momentum distribution")

In [ ]:

    save("Initial_Distributions/" * data_name * ".jld2", "par_dis", u)

In [ ]:
mod(800000 ÷ 10) 